In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import cma

import cirq
import sympy as sp

In [2]:
#Qubits on Square-Octagon Lattice w/ Lattice Parameters
qubits = [0, 1, 2, 3, 4, 5, 6, 7]
Nq = len(qubits)

xx_links = [(0, 1), (2, 3)]
yy_links = [(0, 3), (1, 2)]
yy_links_reversed = [(1, 2), (0, 3)]
zz_links = [(0, 4), (1, 5), (2, 6), (3, 7)]
zz_links_reversed = [(3, 7), (2, 6), (1, 5), (0, 4)]

Jx, Jy, Jz = -1.0/math.sqrt(2), -1.0/math.sqrt(2), -1.0
hX = hY = hZ = 0.05/math.sqrt(3)

In [3]:
#Hamiltonian Construction

#Qubit register for Hamiltonian
qreg = cirq.NamedQubit.range(Nq,prefix="q")

#Constructing Pauli Strings
XX_links_strings = []
for (i,j) in xx_links:
    XX_string = []
    for q in range(Nq):
        if q != i and q != j:
            XX_string.append(cirq.I(qreg[q]))
        else:
            XX_string.append(cirq.X(qreg[q]))
    XX_links_strings.append(XX_string)
    
YY_links_strings = []
for (i,j) in yy_links:
    YY_string = []
    for q in range(Nq):
        if q != i and q != j:
            YY_string.append(cirq.I(qreg[q]))
        else:
            YY_string.append(cirq.Y(qreg[q]))
    YY_links_strings.append(YY_string)

ZZ_links_strings = []
for (i,j) in zz_links:
    ZZ_string = []
    for q in range(Nq):
        if q != i and q != j:
            ZZ_string.append(cirq.I(qreg[q]))
        else:
            ZZ_string.append(cirq.Z(qreg[q]))
    ZZ_links_strings.append(ZZ_string)

X_qubit_strings = []
Y_qubit_strings = []
Z_qubit_strings = []
for k in qubits:
    X_string = []
    Y_string = []
    Z_string = []
    for q in range(Nq):
        if q != k:
            X_string.append(cirq.I(qreg[q]))
            Y_string.append(cirq.I(qreg[q]))
            Z_string.append(cirq.I(qreg[q]))
        else:
            X_string.append(cirq.X(qreg[q]))
            Y_string.append(cirq.Y(qreg[q]))
            Z_string.append(cirq.Z(qreg[q]))
    X_qubit_strings.append(X_string)
    Y_qubit_strings.append(Y_string)
    Z_qubit_strings.append(Z_string)

In [4]:
#Hamiltonian Construction (ctd.)

#Convert Pauli Strings into Products
XX_links_terms = []
for XX_link in XX_links_strings:
    XX_links_terms.append(cirq.PauliString(XX_link))

YY_links_terms = []
for YY_link in YY_links_strings:
    YY_links_terms.append(cirq.PauliString(YY_link))
    
ZZ_links_terms = []
for ZZ_link in ZZ_links_strings:
    ZZ_links_terms.append(cirq.PauliString(ZZ_link))

X_qubit_terms = []
for qubit_string in X_qubit_strings:
    X_qubit_terms.append(cirq.PauliString(qubit_string))

Y_qubit_terms = []
for qubit_string in Y_qubit_strings:
    Y_qubit_terms.append(cirq.PauliString(qubit_string))

Z_qubit_terms = []
for qubit_string in Z_qubit_strings:
    Z_qubit_terms.append(cirq.PauliString(qubit_string))

In [5]:
#Hamiltonian Construction (ctd.)

#Add up all the terms
Hamiltonian = 0
for XX_link in XX_links_terms:
    Hamiltonian += Jx*XX_link
for YY_link in YY_links_terms:
    Hamiltonian += Jy*YY_link
for ZZ_link in ZZ_links_terms:
    Hamiltonian += Jz*ZZ_link
for X_qubit in X_qubit_terms:
    Hamiltonian += hX*X_qubit
for Y_qubit in Y_qubit_terms:
    Hamiltonian += hY*Y_qubit
for Z_qubit in Z_qubit_terms:
    Hamiltonian += hZ*Z_qubit

In [6]:
##Exact Diagonalization of Hamiltonian (Square-Octagon Model)
eigvals, eigvecs = np.linalg.eig(Hamiltonian.matrix())
print(f"Min eigvalue: {min(eigvals)}")

Min eigvalue: (-4.701075608032793-2.1695117316858278e-17j)


In [7]:
##Hamiltonian Variational Ansatz Circuit Construction
def HVA_circuit(alpha, beta, gamma, alpha_mag, beta_mag, gamma_mag, n_layers):
    global xx_links
    global yy_links
    global yy_links_reversed
    global zz_links
    global zz_links_reversed
    global qubits
    #Initialize Circuit
    HVA = cirq.Circuit()
    qreg_HVA = cirq.NamedQubit.range(Nq, prefix="q")
    #Declare variational parameters
    #alpha = sp.symarray('alpha', n_layers)
    #beta = sp.symarray('beta', n_layers)
    #gamma = sp.symarray('gamma', n_layers)
    #alpha_mag = sp.symarray('alpha_mag', n_layers)
    #beta_mag = sp.symarray('beta_mag', n_layers)
    #gamma_mag = sp.symarray('gamma_mag', n_layers)
    #Ansatz Construction
    for idx_layer in range(n_layers):
    #Construct X-terms portion of layer
        HVA.append(cirq.rx(2*alpha_mag[idx_layer]).on(q) for q in qreg_HVA)
        for (i,j) in xx_links:
            HVA.append(cirq.H(qreg_HVA[i]))
            HVA.append(cirq.H(qreg_HVA[j]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
            HVA.append(cirq.rz(2*alpha[idx_layer]).on(qreg_HVA[min(i,j)]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
            HVA.append(cirq.H(qreg_HVA[i]))
            HVA.append(cirq.H(qreg_HVA[j]))
    #Construct Y-terms portion of layer
        HVA.append(cirq.ry(2*beta_mag[idx_layer]).on(q) for q in qreg_HVA)
        for (i,j) in yy_links:
            HVA.append(cirq.rx(math.pi/2).on(qreg_HVA[i]))
            HVA.append(cirq.rx(math.pi/2).on(qreg_HVA[j]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
        for (i,j) in yy_links_reversed:
            HVA.append(cirq.rz(2*beta[idx_layer]).on(qreg_HVA[min(i,j)]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
            HVA.append(cirq.rx(-math.pi/2).on(qreg_HVA[i]))
            HVA.append(cirq.rx(-math.pi/2).on(qreg_HVA[j]))
    #Construct Z-terms portion of layer
        HVA.append(cirq.rz(2*gamma_mag[idx_layer]).on(q) for q in qreg_HVA)
        for (i,j) in zz_links:
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
        for (i,j) in zz_links_reversed:
            HVA.append(cirq.rz(2*gamma[idx_layer]).on(qreg_HVA[min(i,j)]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
    return HVA

In [8]:
##Construct Objective Function
def objective(params, hamiltonian, qreg, simulator):
    alpha = params[:n_layers]
    beta = params[n_layers:2*n_layers]
    gamma = params[2*n_layers:3*n_layers]
    alpha_mag = params[3*n_layers:4*n_layers]
    beta_mag = params[4*n_layers:5*n_layers]
    gamma_mag = params[5*n_layers:]
    ansatz = HVA_circuit(alpha, beta, gamma, alpha_mag, beta_mag, gamma_mag, n_layers)
    #expect = simulator.simulate_expectation_values(ansatz, hamiltonian)
    psi = ansatz.final_state_vector()
    expect = hamiltonian.expectation_from_state_vector(psi, qubit_map={q: i for i, q in enumerate(qreg)})
    return np.real(expect)

In [9]:
##Perform VQE

#Initialize the simulator
simulator = cirq.Simulator()
qreg = cirq.NamedQubit.range(Nq, prefix="q")

#Specify max number of layers
n_layers_max = 6

#Loop that executes VQE (Use CMA-ES Optimizer)
for n_layers in range(1, n_layers_max+1):
    x0 = np.random.uniform(0.0, 2*np.pi, size=6*n_layers)
    x, es = cma.fmin2(objective, x0, sigma0 = 1, args=(Hamiltonian, qreg, simulator))
    print(f"n_layers: {n_layers}")
    es.result_pretty()
    print("*" * 40)

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=952973, Fri May  6 15:27:32 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 -2.849801739600260e+00 1.0e+00 9.22e-01  9e-01  9e-01 0:00.1
    2     18 -3.751447838745394e-01 1.1e+00 8.72e-01  8e-01  9e-01 0:00.1
    3     27 -3.396099856369705e+00 1.3e+00 8.06e-01  7e-01  9e-01 0:00.2
   49    441 -4.673710719837222e+00 4.8e+00 2.99e-02  9e-03  2e-02 0:03.3
  100    900 -4.678242255001587e+00 5.6e+01 2.82e-02  2e-03  8e-02 0:06.6
  176   1584 -4.678564599890183e+00 4.8e+01 4.18e-05  6e-07  2e-05 0:11.7
  200   1800 -4.678564599922380e+00 4.8e+01 4.12e-06  4e-08  1e-06 0:13.3
  206   1854 -4.678564599922555e+00 5.1e+01 2.79e-06  3e-08  8e-07 0:13.7
termination on tolfun=1e-11 (Fri May  6 15:27:46 2022)
final/bestever f-value = -4.678565e+00 -4.678565e+00
incumbent solution: [4.562055871527355, 4.562064500235819, -1.568467627117381, 3.1405650804945267, 4.739497284640797, 5.1073884908849925]


   63    819 -1.403818995341955e+00 1.9e+00 1.04e+00  9e-01  1e+00 0:19.8
   86   1118 -1.009947600654915e+00 1.9e+00 8.43e-01  7e-01  9e-01 0:27.0
  100   1300 -1.296814748762765e+00 2.0e+00 7.77e-01  6e-01  8e-01 0:31.4
  129   1677 -1.406537585522636e+00 2.3e+00 4.38e-01  4e-01  5e-01 0:40.6
  162   2106 -2.598182662215541e+00 2.6e+00 4.70e-01  4e-01  5e-01 0:50.9
  197   2561 -2.789359085402337e+00 2.7e+00 2.18e-01  2e-01  2e-01 1:01.9
  200   2600 -2.869155704173561e+00 2.7e+00 1.92e-01  1e-01  2e-01 1:02.8
  242   3146 -4.358761819516229e+00 2.6e+00 1.04e-01  7e-02  1e-01 1:15.8
  287   3731 -4.592168839442516e+00 2.7e+00 3.18e-02  2e-02  3e-02 1:30.0
  300   3900 -4.631351179275333e+00 2.9e+00 2.92e-02  2e-02  3e-02 1:34.1
  352   4576 -4.671857615502519e+00 3.8e+00 1.14e-02  7e-03  1e-02 1:50.3
  400   5200 -4.679177968954125e+00 5.4e+00 6.97e-03  4e-03  7e-03 2:05.5
  458   5954 -4.682039521009960e+00 1.0e+01 3.70e-03  2e-03  5e-03 2:23.9
  500   6500 -4.682742585799542e+00 1.

  125   1750 -1.253651110720389e+00 2.3e+00 1.34e+00  1e+00  1e+00 0:52.1
  152   2128 -1.993811936981462e+00 2.5e+00 1.15e+00  1e+00  1e+00 1:03.4
  182   2548 -9.224095407193005e-01 2.5e+00 1.12e+00  1e+00  1e+00 1:15.8
  200   2800 -1.235204221082770e+00 2.7e+00 1.10e+00  1e+00  1e+00 1:23.3
  234   3276 -1.781632610870088e+00 2.7e+00 7.96e-01  7e-01  9e-01 1:37.6
  271   3794 -6.209437878060863e-01 2.8e+00 5.20e-01  4e-01  6e-01 1:52.9
  300   4200 -1.446480391851829e+00 2.9e+00 5.80e-01  5e-01  7e-01 2:04.9
  341   4774 -9.284912583787368e-01 3.1e+00 4.75e-01  4e-01  6e-01 2:22.2
  385   5390 -1.262965402588396e+00 3.2e+00 4.54e-01  4e-01  6e-01 2:40.5
  400   5600 -9.531969947326286e-01 3.3e+00 4.47e-01  4e-01  6e-01 2:46.7
  448   6272 -1.394781835213553e+00 3.7e+00 2.96e-01  2e-01  4e-01 3:06.8
  499   6986 -3.218264398508343e+00 3.6e+00 1.59e-01  1e-01  2e-01 3:28.1
  500   7000 -3.379159839231365e+00 3.6e+00 1.57e-01  1e-01  2e-01 3:28.6
  556   7784 -4.421980077260423e+00 3.

 8500 119000 -4.701036675479221e+00 5.8e+02 2.12e-03  5e-06  5e-04 57:18.8
 8600 120400 -4.701036706317664e+00 6.1e+02 2.29e-03  6e-06  5e-04 57:58.6
 8700 121800 -4.701036772425544e+00 6.2e+02 4.63e-03  1e-05  8e-04 58:38.3
 8800 123200 -4.701036796585463e+00 5.4e+02 3.11e-03  6e-06  5e-04 59:17.8
 8900 124600 -4.701036982585125e+00 5.7e+02 1.33e-02  2e-05  2e-03 59:57.5
 9000 126000 -4.701037189976618e+00 5.4e+02 5.86e-03  1e-05  6e-04 60:37.0
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 9100 127400 -4.701037276082967e+00 4.7e+02 5.09e-03  9e-06  4e-04 61:16.5
 9200 128800 -4.701037369334752e+00 5.8e+02 6.71e-03  1e-05  6e-04 61:56.1
 9300 130200 -4.701037438904676e+00 5.6e+02 5.63e-03  7e-06  4e-04 62:35.6
 9400 131600 -4.701037481544259e+00 5.3e+02 6.03e-03  7e-06  3e-04 63:15.0
 9500 133000 -4.701037512630013e+00 5.8e+02 4.58e-03  5e-06  2e-04 63:54.7
 9600 134400 -4.701037535667068e+00 5.8e+02 4.60e-03  5e-06  2e-04 64:34.3
 9700 135800 -4.701037637526

 3800  53200 -4.700915956851873e+00 1.7e+02 8.95e-04  2e-04  3e-03 29:13.7
 3900  54600 -4.700925600876861e+00 1.7e+02 6.63e-04  2e-04  2e-03 29:59.8
 4000  56000 -4.700930614900935e+00 1.7e+02 3.76e-04  8e-05  1e-03 30:46.1
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 4100  57400 -4.700932105496633e+00 1.9e+02 2.33e-04  5e-05  6e-04 31:32.2
 4200  58800 -4.700936302438594e+00 2.1e+02 4.52e-04  1e-04  1e-03 32:18.4
 4300  60200 -4.700939738045032e+00 2.3e+02 6.71e-04  2e-04  2e-03 33:04.5
 4400  61600 -4.700952115238944e+00 2.1e+02 1.20e-03  3e-04  3e-03 33:50.8
 4500  63000 -4.700955761787986e+00 1.7e+02 6.65e-04  1e-04  1e-03 34:36.9
 4600  64400 -4.700957111781761e+00 1.7e+02 4.50e-04  1e-04  8e-04 35:22.9
 4700  65800 -4.700958508682778e+00 1.8e+02 6.06e-04  1e-04  1e-03 36:09.0
 4800  67200 -4.700962649614030e+00 1.9e+02 8.60e-04  2e-04  1e-03 36:55.2
 4900  68600 -4.700967472666935e+00 2.0e+02 9.41e-04  2e-04  2e-03 37:41.2
 5000  70000 -4.700971686866

13800 193200 -4.701039111243954e+00 4.8e+02 8.92e-03  3e-05  1e-03 106:15.8
13900 194600 -4.701039291143562e+00 4.6e+02 5.65e-03  2e-05  6e-04 107:02.2
14000 196000 -4.701039462893595e+00 4.3e+02 5.47e-03  2e-05  5e-04 107:48.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
14100 197400 -4.701039589939261e+00 3.9e+02 5.22e-03  1e-05  4e-04 108:34.8
14200 198800 -4.701039688107648e+00 3.8e+02 4.82e-03  1e-05  4e-04 109:21.4
14300 200200 -4.701039730889661e+00 3.7e+02 4.27e-03  1e-05  3e-04 110:07.7
14400 201600 -4.701039792393502e+00 4.2e+02 6.71e-03  2e-05  5e-04 110:53.9
14500 203000 -4.701039877453602e+00 5.5e+02 6.64e-03  1e-05  6e-04 111:40.3
14600 204400 -4.701039922428931e+00 6.9e+02 4.24e-03  9e-06  5e-04 112:26.5
14700 205800 -4.701039954785451e+00 8.5e+02 4.52e-03  1e-05  6e-04 113:12.8
14800 207200 -4.701039992190664e+00 8.6e+02 4.73e-03  1e-05  6e-04 113:59.1
14900 208600 -4.701040012387599e+00 8.2e+02 3.40e-03  7e-06  4e-04 114:45.5
15000 210000 -4.

23700 331800 -4.701050372578466e+00 8.2e+02 1.48e-01  2e-05  4e-04 182:45.9
23800 333200 -4.701050461396052e+00 8.6e+02 1.95e-01  2e-05  6e-04 183:32.2
23900 334600 -4.701050552750946e+00 7.7e+02 1.86e-01  2e-05  6e-04 184:18.5
24000 336000 -4.701050655904839e+00 6.9e+02 2.01e-01  2e-05  6e-04 185:04.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
24100 337400 -4.701050801076966e+00 6.7e+02 2.89e-01  3e-05  8e-04 185:50.9
24200 338800 -4.701050939333070e+00 5.8e+02 2.57e-01  2e-05  6e-04 186:37.2
24300 340200 -4.701051134452633e+00 6.9e+02 3.96e-01  4e-05  9e-04 187:23.6
24400 341600 -4.701051257366053e+00 6.9e+02 3.05e-01  3e-05  6e-04 188:09.9
24500 343000 -4.701051331867815e+00 6.2e+02 2.15e-01  2e-05  4e-04 188:56.2
24600 344400 -4.701051422904490e+00 6.1e+02 4.21e-01  4e-05  7e-04 189:42.6
24700 345800 -4.701051485958051e+00 5.2e+02 2.61e-01  2e-05  4e-04 190:29.0
24800 347200 -4.701051506445909e+00 4.9e+02 8.70e-02  7e-06  1e-04 191:15.3
24900 348600 -4.